特定キーワードのファイルをフォルダに移動する。ファイル操作
"モデル名”は以下の処理には使われていない。

In [20]:
import os
import shutil

def move_files_by_keyword(target_dir, keywords):
    # 対象ディレクトリ内のすべてのファイルを取得
    files = [f for f in os.listdir(target_dir) if os.path.isfile(os.path.join(target_dir, f))]

    # 各ファイルをキーワードに基づいて適切なフォルダに移動
    for file in files:
        for keyword in keywords:
            if keyword in file:
                # キーワード名のフォルダのパスを取得
                keyword_folder = os.path.join(target_dir, keyword)
                
                # フォルダが存在しない場合は作成
                if not os.path.exists(keyword_folder):
                    os.makedirs(keyword_folder)
                
                # ファイルを移動
                shutil.move(os.path.join(target_dir, file), os.path.join(keyword_folder, file))
                break  # キーワードが見つかったら次のファイルへ

if __name__ == '__main__':
    TARGET_DIR = './csv'  # 対象となるディレクトリのパス
    KEYWORDS = ['expectedPrice','detaildata']  # キーワードのリスト

    move_files_by_keyword(TARGET_DIR, KEYWORDS)


'''
1つのデータフレームに落とし込む。

In [21]:
import os
import pandas as pd
import unicodedata
from datetime import datetime

def combine_csvs_to_df(directory):
    # 指定されたディレクトリ内のCSVファイルのリストを取得
    csv_files = [f for f in os.listdir(directory) if f.endswith('.csv')]
    df_list = []
    
    # 各CSVファイルを読み込み、一つのDataFrameリストに格納
    for csv_file in csv_files:
        df_temp = pd.read_csv(os.path.join(directory, csv_file))
        
        # ファイル名列を追加
        df_temp['filename'] = csv_file
        df_list.append(df_temp)

    # 全てのDataFrameを結合
    combined_df = pd.concat(df_list, ignore_index=True)
    
    return combined_df

# 使い方
directory = './csv/expectedPrice'
df = combine_csvs_to_df(directory)
# filenameを分割して新しい列を作成
df['date'] = df['filename'].apply(lambda x: x.split('_')[0])
df['Carname'] = df['filename'].apply(lambda x: x.split('_')[1].split('_with')[0])
# 'Carname' 列の値を全角から半角に変換し、空白を削除
df['Carname'] = df['Carname'].apply(lambda x: unicodedata.normalize('NFKC', x).replace(' ', ''))
# '総額'がNaNの場合、'価格'の値を'slide'して'slide'の値を'総額'に設定
df.loc[df['総額'].isna(), '総額'] = df.loc[df['総額'].isna(), '価格']
# 小数を含む列を小数点以下第二位まで丸める
for col in df.select_dtypes(include=['float64']).columns:
    df[col] = df[col].round(1)
    
#---------------------------
dftemp=pd.read_csv("./carlist_management.csv")
dftemp = dftemp[['Carname', 'CarMaker']]

# df1とdf2をCarnameを基に左結合
merged_df = pd.merge(df, dftemp, on='Carname', how='left', suffixes=('', '_new'))

# 空白のCarMakerを更新
merged_df['CarMaker'] = merged_df['CarMaker'].fillna(merged_df['CarMaker_new'])

# 不要な列を削除
merged_df.drop(columns='CarMaker_new', inplace=True)

df = merged_df.drop(['predicted_price_zero_mileage'], axis=1)
df = merged_df.sort_values(['Carname' ,'グレード名', '年式','年', 'Mission','date'])

# 重複を削除
df = df.drop_duplicates()

import re
# df['グレード名'] = df['グレード名'].str.replace(r'\s*[（(][Mm][Tt][)）]\s*|\s*[(（]左ハンドル[)）]\s*', '', regex=True)
# グレード名の文字列置換とNaNの処理
df['グレード名'] = df['グレード名'].str.replace(r'\s*[（(][Mm][Tt][)）]\s*|\s*[(（]左ハンドル[)）]\s*', '', regex=True).fillna('-')



新車からの値落ち率をグラフ画像出力
新車からの値さがり率もここで計算。

In [22]:
# 現在の年を取得
current_year = datetime.now().year
# データフレーム df の処理
# 年式の年数を抽出し、新車からの経過年数を計算
df['年式'] = df['年式'].astype(str)
df['年式年数'] = df['年式'].str.extract(r'(\d+)').astype(int)
df['経過年数'] = current_year - df['年式年数']

# 下落率の計算
df['残価率'] = df['価格'] / df['新車価格']

# 経過年数に対する残価率比率の計算
df['残価率比率'] = df['残価率'] / df['経過年数']

# データ処理とプロットリーマップの作成
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d').dt.date
df['年式'] = pd.to_numeric(df['年式'], errors='coerce')

In [ ]:
# import pandas as pd
# import unicodedata
# import re

# # Function to normalize text
# def normalize_text(text):
#     # Check if the input is a string
#     if isinstance(text, str):
#         # Normalize to NFKC form (which converts full-width characters to half-width)
#         text = unicodedata.normalize('NFKC', text)
#         # Remove '(株)', '(有)' in both full-width and half-width forms
#         text = text.replace('(株)', '').replace('（株）', '').replace('（有）', '').replace('(有)', '').replace('有限会社', '').replace('株式会社', '')
#     return text

# # Apply the normalization function to the 'ショップ名' column
# df['ショップ名'] = df['ショップ名'].apply(normalize_text)

# # Extract unique shop names
# carshop = df['ショップ名'].unique().tolist()

In [ ]:
# from difflib import SequenceMatcher

# def similar(a, b):
#     return SequenceMatcher(None, str(a), str(b)).ratio()

# # 店名のリスト（数値が含まれている可能性がある場合）
# stores = carshop

# # すべての要素を文字列に変換
# stores = [str(store) for store in stores]

# # 類似度の閾値
# similarity_threshold = 0.8

# # 類似度に基づく系列店の予測
# predicted_chains_high_threshold = []
# for i in range(len(stores)):
#     for j in range(i+1, len(stores)):
#         if similar(stores[i], stores[j]) > similarity_threshold:
#             predicted_chains_high_threshold.append((stores[i], stores[j]))
#             print(f"予測された系列店: {stores[i]} と {stores[j]}")



In [ ]:
# # Carnameとグレード名による集計
# aggregation = {'残価率': ['mean']}
# grouped_df = df.groupby(['CarMaker','Carname','Mission', 'グレード名','年式', '年']).agg(aggregation)

# # 列名を変更
# grouped_df.columns = ['_'.join(col).strip() for col in grouped_df.columns.values]

車両データに対して定期的にデータ収集しているので、Urlが同じなら同一車両データとみなし。
ピボットにてデータ変動がわかるようにする。
さらに、新車からの根オッチ立wも

In [23]:
# '年式' が 2000 より大きい、かつ 'date' が 2020-01-01 より後のデータをフィルタリング
df_filtered = df[(df['年式'] > 2000) & (df['date'] > pd.Timestamp('2023-10-01'))]

#ここではtreemap用の残価率を計算している。# 最新のレコードを取得　Urlでグループ化した後。
latest_records = df_filtered.sort_values('date', ascending=False).groupby('Url').first().reset_index()

# 各車種に対する集計
carname_grade_count = latest_records.groupby(['CarMaker', 'Carname', 'Mission', 'グレード名', '年']).size().reset_index(name='Count')

# 上位の車種タイプと元のデータセットの結合
top_300_latest_records = pd.merge(carname_grade_count, latest_records, on=['CarMaker', 'Carname', 'Mission', 'グレード名', '年'], how='left')

# 残価率の統計情報を計算,最新日付の情報をもとに計算しようとしたが、、できていない。　2024-1-5
residual_value_stats = top_300_latest_records.groupby(['CarMaker', 'Carname', 'Mission', 'グレード名', '年', '年式', 'Count']).agg(
    Min_残価率=('残価率', 'min'),
    Max_残価率=('残価率', 'max'),
    Mean_残価率=('残価率', 'mean'),
    Median_残価率=('残価率', 'median')
).reset_index().sort_values('Median_残価率', ascending=False)

# 日付で昇順に並び替え
top_300_latest_records = top_300_latest_records.sort_values(by='date')




/var/folders/vy/qcjwsd8d57j08jr7q0pp5plw0000gn/T/ipykernel_4253/3436353095.py:2: FutureWarning:

Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.



In [25]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px

app = dash.Dash(__name__)

# 初期のTreemapを作成する関数
def create_treemap(path_order):
    return px.treemap(
        residual_value_stats, 
        path=path_order, 
        values='Median_残価率',
        color='Median_残価率',
        color_continuous_scale='RdBu',
        range_color=[residual_value_stats['Median_残価率'].min(), 2],
        custom_data=['年式', 'Median_残価率', 'グレード名']
    ).update_traces(
        texttemplate="<br>".join([
            "%{customdata[0]}",
            "残価率: %{customdata[1]:.2f}",
            "%{customdata[2]}"
        ]),
        hovertemplate="<br>".join([
            "年式: %{customdata[0]}",
            "残価率: %{customdata[1]:.2f}",
            "グレード名: %{customdata[2]}"
        ])
    ).update_layout(width=2400, height=1200)

# アプリのレイアウト
app.layout = html.Div([
    # パスの順序を選択するためのドロップダウン
    dcc.Dropdown(
        id='path-order-dropdown',
        options=[{'label': i, 'value': i} for i in ['年式', 'CarMaker', 'Carname', 'Mission', '年', 'グレード名']],
        value=['年式', 'CarMaker', 'Carname', 'Mission', '年', 'グレード名'],
        multi=True
    ),
    
    # Treemapを表示するグラフ
    dcc.Graph(id='treemap-plot', figure=create_treemap(['年式', 'CarMaker', 'Carname', 'Mission', '年', 'グレード名']))
])

# コールバックでTreemapの更新
@app.callback(
    Output('treemap-plot', 'figure'),
    [Input('path-order-dropdown', 'value')]
)
def update_treemap(path_order):
    return create_treemap(path_order)

if __name__ == '__main__':
    app.run_server(debug=True)


#ーーーvisualization1ーーーーーーーーーーーーーー

In [ ]:
# #ーーーvisualization1ーーーーーーーーーーーーーー

# import dash
# from dash import dcc, html, dash_table
# from dash.dependencies import Input, Output
# import plotly.express as px
# import pandas as pd


# top_300_latest_records=df_filtered.sort_values(by='date')

# def customize_legend(fig):
#     # 凡例の位置とフォントサイズの設定
#     fig.update_layout(
#         legend=dict(
#             x=0.9,
#             y=0.95,
#             font=dict(size=9),
#             bgcolor='rgba(255, 255, 255, 0.3)',
#             bordercolor='Black',
#             borderwidth=0.1
#         )
#     )
#     return fig

# #ここでは３つのチャートを用意する。
# #1　treemap で選択した車両の残価率を視覚的に表示する。これはデータを平均化する必要がある。・
# #2 散布図、　x軸に’走行距離'、y軸に価格。　これは全てのデータの視覚的なばらつきを提供する。
# #3 散布図の二つ目、　X軸に年式、
# # Dashアプリケーションの初期化
# app = dash.Dash(__name__)

# def create_selection_component(id, column_name, df, component_type='dropdown', style={'fontSize': 12}, multi=True):
#     unique_values = df[column_name].unique()
#     options = [{'label': i, 'value': i} for i in unique_values]
#     value = [unique_values[0]]  # 初期値をリストとして設定

#     if component_type == 'dropdown':
#         return dcc.Dropdown(
#             id=id,
#             options=options,
#             value=value,
#             style=style,
#             multi=multi
#         )
#     elif component_type == 'checklist':
#         return dcc.Checklist(
#             id=id,
#             options=options,
#             value=value,
#             style=style
#         )

# # 各コンポーネントを作成
# carmaker_dropdown = create_selection_component('carmaker-dropdown', 'CarMaker', top_300_latest_records)
# carname_dropdown = create_selection_component('carname-dropdown', 'Carname', top_300_latest_records)
# modelyear_dropdown = create_selection_component('modelyear-dropdown', '年', top_300_latest_records)
# grade_checklist = create_selection_component('grade-checklist', 'グレード名', top_300_latest_records, 'checklist')


# def create_date_range_slider(df, slider_id):
#     """
#     データフレームの 'date' 列に基づいてRange Sliderを作成する関数。
    
#     :param df: データフレーム
#     :param slider_id: Range SliderのID
#     :return: 設定されたRange Sliderコンポーネント
#     """
#     # 'date' 列を日付型に変換（まだされていない場合）
#     df['date'] = pd.to_datetime(df['date'])

#     # ユニークな日付のリストを作成
#     unique_dates = df['date'].dt.date.unique()
#     unique_dates.sort()  # 日付をソート

#     # 日付を整数のリストに変換
#     date_integers = range(len(unique_dates))

#     # 日付と整数のマッピングを作成
#     date_marks = {i: str(date) for i, date in zip(date_integers, unique_dates)}

#     # Range Sliderの作成
#     return dcc.RangeSlider(
#         id=slider_id,
#         min=0,
#         max=len(unique_dates) - 1,
#         value=[0, len(unique_dates) - 1],
#         marks=date_marks
#     )
# # slider = create_date_range_slider(top_300_latest_records, 'my-date-range-slider')



# #チャートレイアウトに関するコード
# app.layout = html.Div([
#     # サイドバー（全体の20%を占める）
#     html.Div([
#         # サイドバーのコンテンツ
#         html.H3('CarMakerの選択'),
#         carmaker_dropdown,
#         html.H3('Carnameの選択'),
#         carname_dropdown,
#         html.H3('年式の選択'),
#         modelyear_dropdown,
#         html.H3('グレードの選択'),
#         # grade_checklist  # チェックボックスの追加
#         grade_checklist,  # チェックボックスの追加
#         # html.H3('日付の選択'),
#         # # slider        
             
        
#     ], style={'width': '15%', 'float': 'left', 'display': 'inline-block'}),

#     # メインコンテンツエリア（全体の80%を占める）
#     html.Div([
#         # 上部チャートエリア（左右に分割、全体の30%の高さ）
#         html.Div([
#             # 左側のチャートエリア
#             html.Div([
#                 dcc.Graph(id='treemap-plot')
#             ], style={'width': '60%', 'display': 'inline-block', 'height': '100%'}),

#             # 右側のチャートエリア
#             html.Div([
#                 dcc.Graph(id='line-chart'),
#                 html.Div(id='hidden-div-3', style={'display': 'none'})
#             ], style={'width': '40%', 'display': 'inline-block', 'height': '100%'})
#         ], style={'width': '100%', 'height': '35%', 'display': 'inline-block'}),

#         # 下部チャートエリア（左右に分割、全体の70%の高さ）
#         html.Div([
#             # 左側チャート
#             html.Div([
#                 # 散布図に id を追加
#                 dcc.Graph(id='scatter-plot'),
#                 html.Div(id='hidden-div', style={'display': 'none'})  # 隠されたDiv------------
                                 
#             ], style={'width': '50%', 'display': 'inline-block', 'height': '100%'}),

#             # 右側チャート
#             html.Div([
#                 dcc.Graph(id='scatter-plot-2'),
#                 html.Div(id='hidden-div-2', style={'display': 'none'})  # 隠されたDiv------------
                
#             ], style={'width': '50%', 'display': 'inline-block', 'height': '100%'})            
#         ], style={'width': '100%', 'height': '65%','display': 'inline-block'})
        
#     ], style={'width': '85%', 'height': '100vh','float': 'right', 'display': 'inline-block'})
# ], style={'width': '100%', 'display': 'block'})


# # # # CarMaker選択時にCarnameの選択肢を更新  この絞り込みなら動く。
# @app.callback(
#     Output('carname-dropdown', 'options'),
#     [Input('carmaker-dropdown', 'value')]
# )
# def set_carname_options(selected_carmakers):
#     # 選択された Carmaker が None または空のリストの場合、空の選択肢を返す
#     if not selected_carmakers:
#         return []

#     # 選択された Carmaker に基づいて Carname の選択肢をフィルタリング
#     filtered_data = top_300_latest_records[top_300_latest_records['CarMaker'].isin(selected_carmakers)]
#     carnames = [{'label': i, 'value': i} for i in filtered_data['Carname'].unique()]
#     carnames = sorted(carnames, key=lambda x: x['label'])

#     return carnames

# @app.callback(
#     Output('modelyear-dropdown', 'options'),
#     [Input('carmaker-dropdown', 'value'),
#      Input('carname-dropdown', 'value')]
# )
# def set_modelyear_options(selected_carmakers, selected_carnames):
#     # 選択が None の場合は空のリストに置き換える
#     selected_carmakers = selected_carmakers if selected_carmakers is not None else []
#     selected_carnames = selected_carnames if selected_carnames is not None else []

#     # CarMaker と Carname の選択に基づいてデータをフィルタリング
#     filtered_data = top_300_latest_records[
#         top_300_latest_records['CarMaker'].isin(selected_carmakers) &
#         top_300_latest_records['Carname'].isin(selected_carnames)
#     ]

#     # フィルタリングされたデータからユニークな年式のリストを生成
#     modelyears = [{'label': year, 'value': year} for year in filtered_data['年'].unique()]
#     modelyears = sorted(modelyears, key=lambda x: x['label'])
    
#     return modelyears

# @app.callback(
#     Output('grade-checklist', 'options'),
#     [Input('carmaker-dropdown', 'value'),
#      Input('carname-dropdown', 'value'),
#      Input('modelyear-dropdown', 'value')]
# )
# def set_grade_options(selected_carmakers, selected_carnames, selected_modelyears):
#     selected_carmakers = selected_carmakers if selected_carmakers is not None else []
#     selected_carnames = selected_carnames if selected_carnames is not None else []
#     selected_modelyears = selected_modelyears if selected_modelyears is not None else []

#     filtered_data = top_300_latest_records[
#         top_300_latest_records['CarMaker'].isin(selected_carmakers) &
#         top_300_latest_records['Carname'].isin(selected_carnames) &
#         (selected_modelyears == [] or top_300_latest_records['年'].isin(selected_modelyears))
#     ]

#     grades = [{'label': grade, 'value': grade} for grade in sorted(filtered_data['グレード名'].unique())]
#     grades = sorted(grades, key=lambda x: x['label'])

#     return grades

# # データをフィルタリングする関数（これはあなたのコールバックに必要な部分です）  ----- 追加
# def filter_data(selected_carmakers, selected_carnames, selected_modelyears, selected_grades):
#     return top_300_latest_records[
#         (selected_carmakers == [] or top_300_latest_records['CarMaker'].isin(selected_carmakers)) &
#         (selected_carnames == [] or top_300_latest_records['Carname'].isin(selected_carnames)) &
#         (selected_modelyears == [] or top_300_latest_records['年'].isin(selected_modelyears)) &
#         (selected_grades == [] or top_300_latest_records['グレード名'].isin(selected_grades))
#     ]
# #-------------------------------------------------
# # コールバックの定義
# @app.callback(
#     [Output('treemap-plot', 'figure'),
#      Output('line-chart', 'figure'),
#      Output('scatter-plot-2', 'figure'),
#      Output('scatter-plot', 'figure')
#      ],
#     [Input('carmaker-dropdown', 'value'),
#      Input('carname-dropdown', 'value'),
#      Input('modelyear-dropdown', 'value'),
#     #  Input('grade-dropdown', 'value')]
#      Input('grade-checklist', 'value')]
# )

# # def update_charts(selected_carmakers, selected_carnames):
# def update_charts(selected_carmakers, selected_carnames, selected_modelyears, selected_grades):
#     #-----------------
#     global global_filtered_data
#     global_filtered_data = filter_data(selected_carmakers, selected_carnames, selected_modelyears, selected_grades)

#     filtered_data_tree = residual_value_stats[
#         (selected_carmakers == [] or residual_value_stats['CarMaker'].isin(selected_carmakers)) &
#         (selected_carnames == [] or residual_value_stats['Carname'].isin(selected_carnames))&
#         (selected_modelyears == [] or residual_value_stats['年'].isin(selected_modelyears))&
#         (selected_grades == [] or residual_value_stats['グレード名'].isin(selected_grades))
#     ]

#     # トレーマップチャートの更新
#     treemap_figure = px.treemap(
#         filtered_data_tree, 
#         path=['CarMaker', 'Carname', 'Mission', '年', 'グレード名', '年式'], 
#         values='Median_残価率',
#         color='Median_残価率',
#         color_continuous_scale='RdBu',
#         range_color=[residual_value_stats['Median_残価率'].min(), 2],
#         custom_data=['年式', 'Median_残価率']
#         ).update_traces(
#         texttemplate="<br>".join(["%{customdata[0]}", "残価率: %{customdata[1]:.2f}"]),
#         hovertemplate="<br>".join(["年式: %{customdata[0]}", "残価率: %{customdata[1]:.2f}"])
#         )
#         # .update_layout(width=1200, height=600)
        
#         # 散布図2の更新
#     scatter_plot_2 = px.scatter(
#         global_filtered_data,
#         x='predicted_price',
#         y='価格',
#         trendline='ols',
#         color='年式',
#         size='走行距離',  # サイズを「走行距離」に基づいて設定
#         animation_frame="date",
#         symbol='グレード名',
#         custom_data=['Url']  # URLを custom_data に設定
#     )

#     scatter_plot_2 = customize_legend(scatter_plot_2)

#     # X軸とY軸の範囲を同じにする
#     max_range = max(global_filtered_data['predicted_price'].max(), global_filtered_data['価格'].max())
#     scatter_plot_2.update_xaxes(range=[0, max_range])
#     scatter_plot_2.update_yaxes(range=[0, max_range])

#     # 散布図の更新
#     scatter_plot = px.scatter(
#         global_filtered_data,
#         x='走行距離',
#         # marginal_x="violin",
#         y='価格',
#         trendline='ols',
#         color='年式',
#         animation_frame="date",
#         symbol='グレード名',
#         custom_data=['Url'],  # URLを custom_data に設定
        
#         # その他の散布図設定     
#     )
#     scatter_plot = customize_legend(scatter_plot)
    
#     line_chart_figure = px.scatter(
#         global_filtered_data,
#         x='年式',  # 横軸
#         # marginal_x="violin",
#         trendline='ols',
#         y='残価率',  # 縦軸
#         animation_frame="date",
#         range_y=[0.1,1.5],
#         symbol='Mission',
#         size='走行距離',
#         color='グレード名',  # 各車名ごとに異なる色で表示
#          custom_data=['Url']
#     )
#     line_chart_figure = customize_legend(line_chart_figure)
    

#     return [treemap_figure, line_chart_figure, scatter_plot_2, scatter_plot ]
    
# # クライアントサイドのコールバック
# # 散布図1のクリックイベントを処理するクライアントサイドのコールバック
# app.clientside_callback(
#     """
#     function(clickData) {
#         if(clickData) {
#             const url = clickData.points[0].customdata;
#             window.open(url, '_blank');
#         }
#     }
#     """,
#     Output('hidden-div', 'children'),
#     [Input('scatter-plot', 'clickData')]
# )

# # 散布図2のクリックイベントを処理するクライアントサイドのコールバック
# app.clientside_callback(
#     """
#     function(clickData) {
#         if(clickData) {
#             const url = clickData.points[0].customdata;
#             window.open(url, '_blank');
#         }
#     }
#     """,
#     Output('hidden-div-2', 'children'),
#     [Input('scatter-plot-2', 'clickData')]
# )

# # 残価率のクリックイベントを処理するクライアントサイドのコールバック
# app.clientside_callback(
#     """
#     function(clickData) {
#         if(clickData) {
#             const url = clickData.points[0].customdata;
#             window.open(url, '_blank');
#         }
#     }
#     """,
#     Output('hidden-div-3', 'children'),
#     [Input('line-chart', 'clickData')]
# )



# # アプリの実行
# if __name__ == '__main__':
#     app.run_server(debug=True)


style.css   適用版

In [29]:
import dash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd


top_300_latest_records=df_filtered.sort_values(by='date')

def customize_legend(fig):
    # 凡例の位置とフォントサイズの設定
    fig.update_layout(
        legend=dict(
            x=0.9,
            y=0.95,
            font=dict(size=8),
            bgcolor='rgba(255, 255, 255, 0.3)',
            bordercolor='Black',
            borderwidth=0.1
        )
    )
    return fig

#ここでは３つのチャートを用意する。
#1　treemap で選択した車両の残価率を視覚的に表示する。これはデータを平均化する必要がある。・
#2 散布図、　x軸に’走行距離'、y軸に価格。　これは全てのデータの視覚的なばらつきを提供する。
#3 散布図の二つ目、　X軸に年式、
# Dashアプリケーションの初期化
app = dash.Dash(__name__)

def create_selection_component(id, column_name, df, component_type='dropdown', style={'fontSize': 10}, multi=True):
    unique_values = df[column_name].unique()
    options = [{'label': i, 'value': i} for i in unique_values]
    value = [unique_values[0]]  # 初期値をリストとして設定

    if component_type == 'dropdown':
        return dcc.Dropdown(
            id=id,
            options=options,
            value=value,
            style=style,
            multi=multi
        )
    elif component_type == 'checklist':
        return dcc.Checklist(
            id=id,
            options=options,
            value=value,
            style=style
        )

# 各コンポーネントを作成
carmaker_dropdown = create_selection_component('carmaker-dropdown', 'CarMaker', top_300_latest_records)
carname_dropdown = create_selection_component('carname-dropdown', 'Carname', top_300_latest_records)
# modelyear_dropdown = create_selection_component('modelyear-dropdown', '年', top_300_latest_records)
modelyear_checklist = create_selection_component('modelyear-checklist', '年', top_300_latest_records, 'checklist')
grade_checklist = create_selection_component('grade-checklist', 'グレード名', top_300_latest_records, 'checklist')
mission_checklist = create_selection_component('mission-checklist', 'Mission', top_300_latest_records, 'checklist')

#ここはまだ動いてない。　1/10
# def create_date_range_slider(df, slider_id):
#     """
#     データフレームの 'date' 列に基づいてRange Sliderを作成する関数。
    
#     :param df: データフレーム
#     :param slider_id: Range SliderのID
#     :return: 設定されたRange Sliderコンポーネント
#     """
#     # 'date' 列を日付型に変換（まだされていない場合）
#     # df['date'] = pd.to_datetime(df['date'])

#     # ユニークな日付のリストを作成
#     unique_dates = df['date'].dt.date.unique()
#     unique_dates.sort()  # 日付をソート

#     # 日付を整数のリストに変換
#     date_integers = range(len(unique_dates))

#     # 日付と整数のマッピングを作成
#     date_marks = {i: str(date) for i, date in zip(date_integers, unique_dates)}

#     # Range Sliderの作成
#     return dcc.RangeSlider(
#         id=slider_id,
#         min=0,
#         max=len(unique_dates) - 1,
#         value=[0, len(unique_dates) - 1],
#         marks=date_marks
#     )
# slider = create_date_range_slider(top_300_latest_records, 'my-date-range-slider')



#チャートレイアウトに関するコード
#----------------------------------------------
app.layout = html.Div([
    html.Div([  # サイドバー
        # slider,
        html.H3('CarMakerの選択'),
        carmaker_dropdown,
        html.H3('Carnameの選択'),
        carname_dropdown,
        html.H3('年式の選択'),
        modelyear_checklist,
        html.H3('グレードの選択'),
        grade_checklist,
        html.H3('Missionの選択'),
        mission_checklist,
    ], className='sidebar'),

    html.Div([  # メインコンテンツエリア
    # 左側のコンテンツエリア
    html.Div([
        dcc.Graph(id='treemap-plot', className='chart-container'),
        # dcc.Graph(id='scatter-plot', className='chart-container'),
        # html.Div(id='hidden-div', style={'display': 'none'})  # 隠されたDiv------------
        
    ], className='content-left'),

    # 右側のコンテンツエリア
    html.Div([
        dcc.Graph(id='scatter-plot', className='chart-container'),
        html.Div(id='hidden-div', style={'display': 'none'}) , # 隠されたDiv------------
        
        dcc.Graph(id='line-chart', className='chart-container'),
        html.Div(id='hidden-div-3', style={'display': 'none'}),
        dcc.Graph(id='scatter-plot-2', className='chart-container'),
        html.Div(id='hidden-div-2', style={'display': 'none'}),
    ], className='content-right'),
], className='main-content'),
], className='container')  
#----------------------------------------------


def filter_data_by_selection(df, carmakers, carnames, modelyears, grades ,missions):
    # Noneの場合は空のリストに置き換える
    carmakers = [] if carmakers is None else carmakers
    carnames = [] if carnames is None else carnames
    modelyears = [] if modelyears is None else modelyears
    grades = [] if grades is None else grades
    missions = [] if missions is None else missions

    return df[
        (carmakers == [] or df['CarMaker'].isin(carmakers)) &
        (carnames == [] or df['Carname'].isin(carnames)) &
        (modelyears == [] or df['年'].isin(modelyears)) &
        (grades == [] or df['グレード名'].isin(grades)) &
        (missions == [] or df['Mission'].isin(missions))
    ]

@app.callback(
    Output('carname-dropdown', 'options'),
    [Input('carmaker-dropdown', 'value')]
)
def set_carname_options(selected_carmakers):
    filtered_data = filter_data_by_selection(top_300_latest_records, selected_carmakers, [], [], [], [])
    carnames = [{'label': i, 'value': i} for i in sorted(filtered_data['Carname'].unique())]
    return carnames

@app.callback(
    Output('modelyear-checklist', 'options'),
    [Input('carmaker-dropdown', 'value'),
     Input('carname-dropdown', 'value')]
)
def set_modelyear_options(selected_carmakers, selected_carnames):
    filtered_data = filter_data_by_selection(top_300_latest_records, selected_carmakers, selected_carnames, [], [], [])
    modelyears = [{'label': year, 'value': year} for year in sorted(filtered_data['年'].unique())]
    return modelyears

@app.callback(
    Output('grade-checklist', 'options'),
    [Input('carmaker-dropdown', 'value'),
     Input('carname-dropdown', 'value'),
     Input('modelyear-checklist', 'value')]
)
def set_grade_options(selected_carmakers, selected_carnames, selected_modelyears):
    filtered_data = filter_data_by_selection(top_300_latest_records, selected_carmakers, selected_carnames, selected_modelyears, [], [])
    grades = [{'label': grade, 'value': grade} for grade in sorted(filtered_data['グレード名'].unique())]
    return grades

@app.callback(
    Output('mission-checklist', 'options'),
    [Input('carmaker-dropdown', 'value'),
     Input('carname-dropdown', 'value'),
     Input('modelyear-checklist', 'value'),
     Input('grade-checklist', 'value')]
)
def set_mission_options(selected_carmakers, selected_carnames, selected_modelyears, selected_grades):
    filtered_data = filter_data_by_selection(top_300_latest_records, selected_carmakers, selected_carnames, selected_modelyears, selected_grades,[])
    missions = [{'label': mission, 'value': mission} for mission in sorted(filtered_data['Mission'].unique())]
    return missions


@app.callback(
    [Output('treemap-plot', 'figure'),
     Output('line-chart', 'figure'),
     Output('scatter-plot-2', 'figure'),
     Output('scatter-plot', 'figure')],
    [Input('carmaker-dropdown', 'value'),
     Input('carname-dropdown', 'value'),
     Input('modelyear-checklist', 'value'),
     Input('grade-checklist', 'value'),
     Input('mission-checklist', 'value')]
)

# # # CarMaker選択時にCarnameの選択肢を更新  この絞り込みなら動く。


def update_charts(selected_carmakers, selected_carnames, selected_modelyears, selected_grades, selected_missions):
    global global_filtered_data
    global_filtered_data = filter_data_by_selection(top_300_latest_records, selected_carmakers, selected_carnames, selected_modelyears, selected_grades, selected_missions)

    filtered_data_tree = residual_value_stats[
        (selected_carmakers == [] or residual_value_stats['CarMaker'].isin(selected_carmakers)) &
        (selected_carnames == [] or residual_value_stats['Carname'].isin(selected_carnames))&
        (selected_modelyears == [] or residual_value_stats['年'].isin(selected_modelyears))&
        (selected_grades == [] or residual_value_stats['グレード名'].isin(selected_grades))
    ]

    # トレーマップチャートの更新
    treemap_figure = px.treemap(
        filtered_data_tree, 
        path=['CarMaker', 'Carname', 'Mission', '年', 'グレード名', '年式'], 
        values='Median_残価率',
        color='Median_残価率',
        color_continuous_scale='RdBu',
        range_color=[residual_value_stats['Median_残価率'].min(), 2],
        custom_data=['年式', 'Median_残価率']
        ).update_traces(
        texttemplate="<br>".join(["%{customdata[0]}", "残価率: %{customdata[1]:.2f}"]),
        hovertemplate="<br>".join(["年式: %{customdata[0]}", "残価率: %{customdata[1]:.2f}"])
        )
        # .update_layout(width=1200, height=600)
    
    # sunburst_figure = px.sunburst(
    #     filtered_data_tree, 
    #     path=['CarMaker', 'Carname', 'Mission', '年', 'グレード名', '年式'],  # 階層のパス
    #     values='Median_残価率',  # 各セグメントのサイズを決定する値
    #     color='Median_残価率',   # セグメントの色
    #     # color_continuous_scale='RdBu',  # 色のスケール
    #     range_color=[residual_value_stats['Median_残価率'].min(), 2],  # 色の範囲
    #     custom_data=['年式', 'Median_残価率']  # カスタムデータ
    #     ).update_traces(
    #     texttemplate="<br>".join(["%{customdata[0]}", "残価率: %{customdata[1]:.2f}"]),  # テキストテンプレート
    #     hovertemplate="<br>".join(["年式: %{customdata[0]}", "残価率: %{customdata[1]:.2f}"])  # ホバーテンプレート
    #     )
            
        # 散布図2の更新
    scatter_plot_2 = px.scatter(
        global_filtered_data,
        x='predicted_price',
        y='価格',
        trendline='ols',
        color='年式',
        size='走行距離',  # サイズを「走行距離」に基づいて設定
        animation_frame="date",
        symbol='グレード名',
        custom_data=['Url']  # URLを custom_data に設定
    )

    scatter_plot_2 = customize_legend(scatter_plot_2)

    # X軸とY軸の範囲を同じにする
    max_range = max(global_filtered_data['predicted_price'].max(), global_filtered_data['価格'].max())
    scatter_plot_2.update_xaxes(range=[0, max_range])
    scatter_plot_2.update_yaxes(range=[0, max_range])

    # # 散布図の更新
    scatter_plot = px.scatter(
        global_filtered_data,
        x='走行距離',
        # marginal_x="violin",
        y='価格',
        trendline='ols',
        color='年式',
        animation_frame="date",
        symbol='グレード名',
        custom_data=['Url'],  # URLを custom_data に設定
        
        # その他の散布図設定     
    )
    scatter_plot = customize_legend(scatter_plot)
    
    
    
    line_chart_figure = px.scatter(
        global_filtered_data,
        x='年式',  # 横軸
        # marginal_x="violin",
        trendline='ols',
        y='残価率',  # 縦軸
        animation_frame="date",
        range_y=[0.1,1.5],
        symbol='Mission',
        size='走行距離',
        color='グレード名',  # 各車名ごとに異なる色で表示
         custom_data=['Url']
    )
    line_chart_figure = customize_legend(line_chart_figure)
    

    return [treemap_figure, line_chart_figure, scatter_plot_2, scatter_plot ]
    # return [treemap_figure, line_chart_figure, scatter_plot_2, scatter_ternary ]

    
# クライアントサイドのコールバック
# 散布図1のクリックイベントを処理するクライアントサイドのコールバック
app.clientside_callback(
    """
    function(clickData) {
        if(clickData) {
            const url = clickData.points[0].customdata;
            window.open(url, '_blank');
        }
    }
    """,
    Output('hidden-div', 'children'),
    [Input('scatter-plot', 'clickData')]
)

# 散布図2のクリックイベントを処理するクライアントサイドのコールバック
app.clientside_callback(
    """
    function(clickData) {
        if(clickData) {
            const url = clickData.points[0].customdata;
            window.open(url, '_blank');
        }
    }
    """,
    Output('hidden-div-2', 'children'),
    [Input('scatter-plot-2', 'clickData')]
)

# 残価率のクリックイベントを処理するクライアントサイドのコールバック
app.clientside_callback(
    """
    function(clickData) {
        if(clickData) {
            const url = clickData.points[0].customdata;
            window.open(url, '_blank');
        }
    }
    """,
    Output('hidden-div-3', 'children'),
    [Input('line-chart', 'clickData')]
)



# アプリの実行
if __name__ == '__main__':
    app.run_server(debug=True)

---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
File /opt/homebrew/lib/python3.11/site-packages/pandas/core/indexes/base.py:3802, in Index.get_loc(
    self=Index(['名称', '年式', 'グレード名', '年', '色', 'Class_Cat...', '経過年数', '残価率', '残価率比率'],
      dtype='object'),
    key=True,
    method=None,
    tolerance=None
)
   3801 try:
-> 3802     return self._engine.get_loc(casted_key)
        casted_key = True
        self = Index(['名称', '年式', 'グレード名', '年', '色', 'Class_Category', 'Url', 'Mission',
       '修復歴', 'difference', '新車価格', '価格', '総額', 'predicted_price', '走行距離',
       'predicted_price_zero_mileage', 'CarMaker', 'filename', 'モデル名', 'ショップ名',
       '排気量', 'date', 'Carname', '年式年数', '経過年数', '残価率', '残価率比率'],
      dtype='object')
   3803 except KeyError as err:

File /opt/homebrew/lib/python3.11/site-packages/pandas/_libs/index.pyx:138, in pandas._libs.index.IndexEngine.get_loc()

File /op

difference を算出するための特徴量。


In [ ]:
# 車種ごとにグループ化し、各グループの最新の日付を取得
latest_dates = df.groupby('Carname')['date'].max()

# 最新の日付に該当する行を取得
latest_data = df[df.apply(lambda row: row['date'] == latest_dates[row['Carname']], axis=1)]

In [ ]:
#ピボット集計。
import numpy as np
import os
from datetime import datetime
import unicodedata
import pandas as pd
import xlsxwriter
# from openpyxl import Workbook
from openpyxl.worksheet.table import Table, TableStyleInfo
from openpyxl import load_workbook

# 今日の年を取得
current_year = datetime.now().year

def get_first_last_nonzero(series):
    non_zero_values = series.replace(0, np.nan).dropna()
    if not non_zero_values.empty:
        return non_zero_values.iloc[0], non_zero_values.iloc[-1]
    else:
        return float('nan'), float('nan')
    
def get_carmaker(df):
    carmaker = df['CarMaker'].iloc[0]
    return str(carmaker) if not pd.isnull(carmaker) else "Unknown"

def create_pivot_table(df):
    return df.pivot_table(
        index=['年', 'グレード名', 'Mission', '年式', '修復歴', '色', '走行距離', 'Url', '新車価格'],
        # columns='date', values=['価格', 'predicted_price'], aggfunc='sum', fill_value=np.nan
        columns='date', values='価格', aggfunc='sum', fill_value=np.nan
    )

def calculate_price_changes(pivot_df):
    pivot_df['最初の非ゼロ価格'], pivot_df['最後の非ゼロ価格'] = zip(*pivot_df.apply(get_first_last_nonzero, axis=1))
    pivot_df['変動額'] = (pivot_df['最後の非ゼロ価格'] - pivot_df['最初の非ゼロ価格']).round(1)
    pivot_df['価格変動種別'] = np.where(
        pivot_df['最後の非ゼロ価格'] > pivot_df['最初の非ゼロ価格'], '値上げ',
        np.where(pivot_df['最後の非ゼロ価格'] < pivot_df['最初の非ゼロ価格'], '値下げ', '変動なし')
    )
    return pivot_df

def save_to_csv(df, carname, carmaker):
    today = datetime.now().strftime('%Y-%m-%d')
    directory_path = os.path.join(today, carmaker)
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)

    csv_filename = f"{carname}_data.csv"
    full_path = os.path.join(directory_path, csv_filename)
    df.to_csv(full_path, encoding='utf-8-sig')
    print(f"File '{csv_filename}' has been exported to {full_path}")

def export_car_data(carname, df):
    current_year = datetime.now().year
    car_df = df[df['Carname'] == carname]

    carmaker = get_carmaker(car_df)
    pivot_df_car = create_pivot_table(car_df)
    pivot_df_car = calculate_price_changes(pivot_df_car)

    pivot_df_car.reset_index(inplace=True)
    pivot_df_car['新車価格'] = pivot_df_car['新車価格'].astype(float)
    pivot_df_car['新車からの下落率'] = ((pivot_df_car['新車価格'] - pivot_df_car['最後の非ゼロ価格']) / pivot_df_car['新車価格'] * 100).round(2)
    pivot_df_car['年式'] = pivot_df_car['年式'].astype(float).astype(int)
    pivot_df_car['車の年齢'] = current_year - pivot_df_car['年式']
    pivot_df_car['年間下落率'] = (pivot_df_car['新車からの下落率'] / pivot_df_car['車の年齢']).round(2)

    pivot_df_car.drop(['最初の非ゼロ価格', '最後の非ゼロ価格'], axis=1, inplace=True)
    
    
    # latest_data と pivot_df を 'Url' を共通キーとしてマージする　　--------　12/25
    # latest_data から 'Url' と 'predicted_price' 列を選択
    latest_data_reduced = latest_data[['Url', 'predicted_price','date']]
    # merged_df = pd.merge(pivot_df_car,latest_data_reduced,  on='Url', how='inner')
    merged_df = pd.merge(pivot_df_car,latest_data_reduced,  on='Url', how='left')
    
    # sorted_df = pivot_df_car.sort_values(by=['変動額'], ascending=[True])
    sorted_df = merged_df.sort_values(by=['Url'], ascending=[True])

    save_to_csv(sorted_df, carname, carmaker)


#     # excel_filename = f"{name}_price_withtable.xlsx"
#     # full_excelpath = os.path.join(today, excel_filename)
#     # # save_df_as_excel_table(full_path,full_excelpath,"sales")
    
# def save_df_as_excel_table(csv_path, excel_path, sheet_name='Sheet1'):
#     # CSVファイルをDataFrameとして読み込む
#     df = pd.read_csv(csv_path)

#     # Excelファイルを作成
#     writer = pd.ExcelWriter(excel_path, engine='openpyxl')
#     df.to_excel(writer, sheet_name=sheet_name, index=False)

#     # ワークブックとシートを取得
#     workbook = writer.book
#     worksheet = writer.sheets[sheet_name]

#     # テーブルの範囲を指定
#     table_range = f"A1:{chr(65 + len(df.columns) - 1)}{len(df) + 1}"

#     # テーブルを作成
#     table = Table(displayName="Table1", ref=table_range)

#     # テーブルスタイルを設定（オプション）
#     style = TableStyleInfo(name="TableStyleMedium9", showFirstColumn=False,
#                            showLastColumn=False, showRowStripes=True, showColumnStripes=True)
#     table.tableStyleInfo = style

#     # テーブルをワークシートに追加
#     worksheet.add_table(table)

#     # ファイルを保存
#     writer.save()


def normalize_string(s):
    return unicodedata.normalize('NFKC', s)

today = datetime.today().strftime('%Y-%m-%d')
if not os.path.exists(today):
    os.mkdir(today)

df['Carname'] = df['Carname'].apply(normalize_string)
unique_names = df['Carname'].drop_duplicates().tolist()

for name in unique_names:
    export_car_data(name, df)
    # Save the DataFrame as an Excel table


In [ ]:
# # 'CarMaker' が NaN のレコードをフィルタリング
# nan_carmaker_records = df[df['CarMaker'].isna()]

# # 結果を表示
# print(nan_carmaker_records)

In [ ]:
# pivot_df = df.pivot_table(index='Carname', columns='date', values='filename', aggfunc='count', fill_value=0)
# display(pivot_df.T)
# pivot_df.T.to_csv("collectStatus.csv")

スコアリングに挑戦
-------------------------------------


In [ ]:
import numpy as np

pivot_df_car2 = df.pivot_table(
        index=['CarMaker','Carname','年','グレード名','新車価格']
        ,columns='年式', values='残価率', aggfunc='mean', fill_value=np.nan)

# 結果の小数点以下を第二位までに丸める
pivot_df_car2 = pivot_df_car2.round(2)

# 年式が2010年以上の列のみを選択
filtered_pivot_df = pivot_df_car2.loc[:, pivot_df_car2.columns >= 2010]

# 列を降順に並べ替え
filtered_pivot_df = filtered_pivot_df.sort_index(axis=1, ascending=False)

# import plotly.express as px
filtered_pivot_df.to_csv("残価率_.csv")


# ヒートマップの作成
fig = px.imshow(
    filtered_pivot_df,
    labels=dict(x="年式", y="車種情報", color="残価率"),
    x=filtered_pivot_df.columns,
    y=filtered_pivot_df.index
)

# ヒートマップのタイトルと軸ラベルを設定
fig.update_layout(
    title="車種ごとの年式別残価率",
    xaxis_title="年式",
    yaxis_title="車種情報",
)

# ヒートマップの表示
fig.show()

In [ ]:
# import dash
# import dash_core_components as dcc
# import dash_html_components as html
# from dash.dependencies import Input, Output
# import plotly.express as px
# import pandas as pd

# # サンプルデータの用意
# df = pd.DataFrame({'category': ['A', 'B', 'C', 'A', 'B', 'C'],
#                    'sub_category': ['A1', 'B1', 'C1', 'A2', 'B2', 'C2'],
#                    'value': [10, 25, 15, 40, 50, 60],
#                    'x': [1, 2, 3, 4, 5, 6],
#                    'y': [6, 5, 4, 3, 2, 1]})

# # Dashアプリの初期化
# app = dash.Dash(__name__)

# # アプリのレイアウト
# app.layout = html.Div([
#     dcc.Graph(
#         id='treemap',
#         figure=px.treemap(df, path=['category', 'sub_category'], values='value')
#     ),
#     dcc.Graph(id='scatter-plot')
# ])

# @app.callback(
#     Output('scatter-plot', 'figure'),
#     [Input('treemap', 'clickData')]
# )
# def update_scatter(clickData):
#     # clickDataがNoneでないことと、'points'キーが含まれていることを確認
#     if clickData and 'points' in clickData:
#         # 'points'リストが空でないことを確認
#         if len(clickData['points']) > 0:
#             clicked_point = clickData['points'][0]

#             # 'id' キーが存在するかどうかをチェック
#             if 'id' in clicked_point:
#                 path = clicked_point['id'].split('/')
#                 filtered_df = df[df['category'] == path[1]]
#                 if len(path) > 2:
#                     filtered_df = filtered_df[filtered_df['sub_category'] == path[2]]
#             else:
#                 # 'id' キーがない場合の代替処理
#                 filtered_df = df
#         else:
#             # 'points'リストが空の場合は散布図を空にする
#             return px.scatter()
#     else:
#         # clickDataがNoneまたは'points'キーがない場合は散布図を空にする
#         return px.scatter()

#     return px.scatter(filtered_df, x='x', y='y')



# # アプリの実行
# if __name__ == '__main__':
#     app.run_server(debug=True)
